# Bot for trading BTCUSDT with crossing moving averages


### THIS BOT IS GOING TO OPERATE WITH CROSSING MOVING AVERAGE LINES WHEN:

#### PRICE > MA5 > MA10 > MA20

### Installs

In [1]:
#%pip install python-binance 
#%pip install numpy
#%pip install pandas

### Imports

In [2]:
import config
import math
from binance.client import Client
from binance.enums import *
import time
import numpy as np
from IPython.display import clear_output

cliente = Client(config.API_KEY, config.API_SECRET, tld='com')


# Definition of coin and parameters

In [3]:
simbolo = 'ETHUSDT'   #Name of specific Symbol we are going to trade

simboloBalance = 'ETH'

quantityOrder_USDT = 15 # Quantity of USDT I am going to buy per operation. Around 15$...

decimales = '{:.2f}' # Number of decimals in buy parameters, if there is too many decimals depending on the cryptocurrency there will be an error of PRICEFILTER 

decimals = 4

# Functions

## ***** MOVING AVERAGES WITH 5 MINS CANDLESTICKS ******



### Moving Average 5 in periods of 5mins during 25 minutes

In [4]:
def _ma5_():

    ma5_local = 0
    sum = 0

    klines = cliente.get_historical_klines(simbolo, Client.KLINE_INTERVAL_5MINUTE, "25 minute ago UTC")

    if(len(klines)==5):
        for i in range (0,5):
            sum = sum +float(klines[i][4]) # 4 value makes reference to the closing price of a candlestick
        
        ma5_local = sum / 5

    return ma5_local

### Moving Average 10 in periods of 5mins during 50 minutes

In [5]:
def _ma10_():

    ma10_local = 0
    sum = 0

    klines = cliente.get_historical_klines(simbolo, Client.KLINE_INTERVAL_5MINUTE, "50 minute ago UTC")

    if(len(klines)==10):
        for i in range (0,10):
            sum = sum +float(klines[i][4]) # 4 value makes reference to the closing price of a candlestick
        
        ma10_local = sum / 10

    return ma10_local

### Moving Average 20 in periods of 5mins during 100 minutes

In [6]:
def _ma20_():

    ma20_local = 0
    sum = 0

    klines = cliente.get_historical_klines(simbolo, Client.KLINE_INTERVAL_5MINUTE, "100 minute ago UTC")

    if(len(klines)==20):
        for i in range (0,20):
            sum = sum +float(klines[i][4]) # 4 value makes reference to the closing price of a candlestick
        
        ma20_local = sum / 20

    return ma20_local

### Function to round down decimals

In [7]:
def round_decimals_down(number:float, decimals:int=4):
    """
    Returns a value rounded down to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more")
    elif decimals == 0:
        return math.floor(number)

    factor = 10 ** decimals
    return math.floor(number * factor) / factor

### *** Loop operation for continuous trading ***

In [ ]:
count = 0
while 1:
    
    for i in range(10):
        if count == 3:
            clear_output(wait=True)
            count = 0
    count += 1
    
    requestMinQtOrder = cliente.get_symbol_info(simbolo)
    ordenes = cliente.get_open_orders(symbol=simbolo)
    print("")
    print("-----------------------------------------------------")
    print("Actual open orders") # si devuelve [] esta vacio
    print(ordenes)
    print("")
    
    quantityOrder = 0
    account = cliente.get_account()
    balances = account['balances']
    for i in balances:
        if simboloBalance in i['asset']:
            quantityOrder += float(i['locked'])

    quantityOrder_left = round_decimals_down(quantityOrder, decimals)
    
   
    if(len(ordenes) != 0 ):
        print('There is ' +str(len(ordenes)) + ' orders opened, no new buys')
        print("Quantity to sell   " + str(quantityOrder_left))
        print("Sell Price if it goes DOWN   " + ordenes[0]['price'])
        print("Sell Price if it goes UP   " + ordenes[1]['price'])
        print("-----------------------------------------------------")
        print("")
        time.sleep(20) #mando el robot a dormir porque EN TEORIA abrio un orden, dejamos que el mercado opere.
        continue
        
    # I obtain the price of the cryptocurrency I am going to trade and transform quantityOrder_USDT into quantityOrder
    list_of_tickers = cliente.get_all_tickers()
    for tick_2 in list_of_tickers:
        if tick_2['symbol'] == simbolo:
            symbolPrice = float(tick_2['price'])
            quantityOrder = round_decimals_down(quantityOrder_USDT/symbolPrice , decimals)
    # END of obtaining price and quantityOrder.
    
    
    ma5  = _ma5_()
    ma10 = _ma10_()
    ma20 = _ma20_()

    if(ma20 == 0 ): continue

    minQtOrder = float(requestMinQtOrder['filters'][1]['minQty'])

    if (minQtOrder !=1 ):
        print("Order accepts decimals")
        print("")
        order_local = '{:.8f}'.format(quantityOrder*0.999)
    else:
        print("Order accepts only integer numbers")
        print("")
        order_local = '{:.0f}'.format(quantityOrder*0.999)
    
    # importante acomodar los decimales de la moneda porque arroja Error Price Filter.

    print("--------" + simbolo + "---------")
    print(" Actual price of " + simbolo + " is: " + str(decimales.format(symbolPrice))) #el .8 es la cantidad de decimales que no trae el simbolo 
    print("")
    print("*******************************")
    print(" Price to buy " + str(decimales.format(ma20*0.995)))
    print(" Price MA5 " + str(decimales.format(ma5)))
    print(" Price MA10 " + str(decimales.format(ma10)))
    print(" Price MA20 " + str(decimales.format(ma20)))
    print("*******************************\n")


    if ( symbolPrice > ma5 and ma5 > ma10 and ma10 > ma20):
        print("Buying if there is no open orders")
        print("")
    
    # ORDENES DE PRUEBA 
        #order = cliente.create_test_order(
        #symbol = simbolo,
        #side = SIDE_BUY,
        #type = ORDER_TYPE_LIMIT,
        #timeInForce = TIME_IN_FORCE_GTC,
        #quantity = cantidadOrden*0.999,
        #price = str(decimales.format(symbolPrice*1.02)),
        #)
#
        #orders = cliente(symbol=simbolo)
        #print(orders)

        order = cliente.order_market_buy(
            symbol = simbolo,
            quantity = quantityOrder
        )
        
        quantityOrder = 0
        account = cliente.get_account()
        balances = account['balances']
        for i in balances:
            if simboloBalance in i['asset']:
                quantityOrder += float(i['free'])

        quantityOrder_left = round_decimals_down(quantityOrder, decimals)

        time.sleep(5)

        #Pongo orden OCO
        print("")
        print("PLACING OCO ORDER")
        print("StopLimitPrice >   " +str(decimales.format(symbolPrice*0.985)),)
        print("Quantity >   " + str(quantityOrder_left))
        print("StopPrice >   " + str(decimales.format(symbolPrice*0.99)))
        print("Price >   " + str(decimales.format(symbolPrice*1.01)))
        print("")
        print("-----------------------------------------------------")
        print("")
        
        ordenOCO = cliente.create_oco_order(
            symbol = simbolo,
            side = SIDE_SELL,
            stopLimitPrice = str(decimales.format(symbolPrice*0.985)),
            stopLimitTimeInForce = TIME_IN_FORCE_GTC,
            ## Error  LOT SIZE es porque no soporta decimales en quantity
            quantity = quantityOrder_left, # BINANCE cobra un fee, tarifa. Sino va a tirar un error de insuficent FOUNDS. O Error LOT SIZE. 
            stopPrice = str(decimales.format(symbolPrice*0.99)),
            price = str(decimales.format(symbolPrice*1.01)), 
            )
    
        time.sleep(20) #mando el robot a dormir porque EN TEORIA abrio un orden, dejamos que el mercado opere.

    else: 
        print("Conditions are not met")
        print("-----------------------------------------------------")
        print("")
        time.sleep(20) #mando el robot a dormir porque EN TEORIA abrio un orden, dejamos que el mercado opere.
    
    # FIN ordenes de prueba

# corregir ma20
# eliminar import BTC primera linea
# corregir filter por FILTERS
# corregir identado en ordenes de prueba


-----------------------------------------------------
Actual open orders
[{'symbol': 'ETHUSDT', 'orderId': 11997712425, 'orderListId': 79518586, 'clientOrderId': 'v0h6WNA0hM6QbCFlBu6VL7', 'price': '1267.89000000', 'origQty': '0.01160000', 'executedQty': '0.00000000', 'cummulativeQuoteQty': '0.00000000', 'status': 'NEW', 'timeInForce': 'GTC', 'type': 'STOP_LOSS_LIMIT', 'side': 'SELL', 'stopPrice': '1274.33000000', 'icebergQty': '0.00000000', 'time': 1671098126963, 'updateTime': 1671098126963, 'isWorking': False, 'workingTime': -1, 'origQuoteOrderQty': '0.00000000', 'selfTradePreventionMode': 'NONE'}, {'symbol': 'ETHUSDT', 'orderId': 11997712426, 'orderListId': 79518586, 'clientOrderId': 'uXVUm8af2rVsdhCL1FEq0o', 'price': '1300.07000000', 'origQty': '0.01160000', 'executedQty': '0.00000000', 'cummulativeQuoteQty': '0.00000000', 'status': 'NEW', 'timeInForce': 'GTC', 'type': 'LIMIT_MAKER', 'side': 'SELL', 'stopPrice': '0.00000000', 'icebergQty': '0.00000000', 'time': 1671098126963, 'upda

# DEBUGGING


In [ ]:
def _ma5_():

    ma5_local = 0
    sum = 0

    klines = cliente.get_historical_klines(simbolo, Client.KLINE_INTERVAL_5MINUTE, "25 minute ago UTC")

    if(len(klines)==5):
        for i in range (0,5):
            sum = sum +float(klines[i][4]) # 4 precio de cierre de la vela
        
        ma5_local = sum / 5

    return ma5_local

def _ma10_():

    ma10_local = 0
    sum = 0

    klines = cliente.get_historical_klines(simbolo, Client.KLINE_INTERVAL_5MINUTE, "50 minute ago UTC")

    if(len(klines)==10):
        for i in range (0,10):
            sum = sum +float(klines[i][4]) # 4 precio de cierre de la vela
        
        ma10_local = sum / 10

    return ma10_local

def _ma20_():

    ma20_local = 0
    sum = 0

    klines = cliente.get_historical_klines(simbolo, Client.KLINE_INTERVAL_5MINUTE, "100 minute ago UTC")

    if(len(klines)==20):
        for i in range (0,20):
            sum = sum +float(klines[i][4]) # 4 precio de cierre de la vela
        
        ma20_local = sum / 20

    return ma20_local

ma5  = _ma5_()
ma10 = _ma10_()
ma20 = _ma20_()

print(ma5 , ma10, ma20)

In [ ]:
requestMinQtOrder = cliente.get_symbol_info(simbolo)

requestMinQtOrder

int(math.floor(float(requestMinQtOrder['filters'][1]['minQty'])))

In [ ]:
account = cliente.get_account()

account['balances']

In [ ]:
#balances = cliente.get_account()
quantityOrder = 0
account = cliente.get_account()
balances = account['balances']
for i in balances:
    if simboloBalance in i['asset']:
        quantityOrder += float(i['free'])
        
quantityOrder = round_decimals_down(quantityOrder, decimals)

quantityOrder

In [ ]:
quantityOrder

In [ ]:
round_decimals_down(quantityOrder, 4)